In [23]:
import os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from zipline.utils.factory import load_bars_from_yahoo
import pytz
import numpy as np
from sklearn import model_selection
from sklearn import preprocessing
import matplotlib.pyplot as plt
import ystockquote

In [88]:
symbols = ['A', 'AA', 'AAP']#, 'ABBV', 'ABT', 'ACN', 'ADBE', 'AES', 'AET', 'AFL']
start = datetime(2000, 1, 1, 0, 0, 0, 0, pytz.utc)
end = datetime(2016, 4, 30, 0, 0, 0, 0, pytz.utc)
dates = pd.date_range(start,end)

In [89]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30'],
              dtype='datetime64[ns, UTC]', length=5965, freq='D')

In [90]:
dates = pd.date_range('2011-10-13', '2016-10-12')
dates

DatetimeIndex(['2011-10-13', '2011-10-14', '2011-10-15', '2011-10-16',
               '2011-10-17', '2011-10-18', '2011-10-19', '2011-10-20',
               '2011-10-21', '2011-10-22',
               ...
               '2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
               '2016-10-07', '2016-10-08', '2016-10-09', '2016-10-10',
               '2016-10-11', '2016-10-12'],
              dtype='datetime64[ns]', length=1827, freq='D')

In [101]:
def get_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    data = []
    for symbol in symbols:
        df_temp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan'])
        df_comp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, na_values=['nan'])
        data.append(df_comp)
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        #print df_temp
        df = df.join(df_temp)
    return [df, data]

In [117]:
[df_old, data] = get_data(symbols, dates)

In [127]:
df_old.isnull().values.any()

True

In [128]:
df_old.isnull().sum().sum()

1707

In [119]:
len(dates)

1827

In [94]:
def get_stocks_info(symbol):
    dict_stocks = ystockquote.get_historical_prices(symbol, '2011-10-13', '2016-10-12')
    json_str = json.dumps(dict_stocks)
    df =pd.read_json(json_str,orient='index')
    df = df.reindex(df.index.rename('Date'))
    return df

In [97]:
def fill_missing(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

In [70]:
jso['2014-06-30']

{'Adj Close': '584.669983',
 'Close': '584.669983',
 'High': '587.710022',
 'Low': '583.289978',
 'Open': '586.76001',
 'Volume': '1478900'}

In [74]:
# jso = {'Date':jso}

In [76]:
df =pd.read_json(json.dumps(jso),orient='index')

In [81]:
df = df.reindex(df.index.rename('Date'))

In [112]:
df = get_stocks_info('A')

In [104]:
fill_missing(df)

In [113]:
new_df = df.filter(['Date','Adj Close'], axis=1)

In [114]:
new_df.isnull().values.any()

False

In [115]:
new_df

,Adj Close
Date,
2011-10-13,22.067759
2011-10-14,22.659041
2011-10-17,22.475540
2011-10-18,22.754190
2011-10-19,22.421169
2011-10-20,22.924099
2011-10-21,23.970738
2011-10-24,25.513509
2011-10-25,24.419297


In [130]:
s = new_df.reindex(dates, fill_value=np.nan)

In [131]:
s

,Adj Close
2011-10-13,22.067759
2011-10-14,22.659041
2011-10-15,NaN
2011-10-16,NaN
2011-10-17,22.475540
2011-10-18,22.754190
2011-10-19,22.421169
2011-10-20,22.924099
2011-10-21,23.970738
2011-10-22,NaN


In [132]:
s.isnull().values.any()

True

In [134]:
s.isnull().sum().sum()*3

1707